In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-z2t1gg71
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-z2t1gg71
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=ad814e5a8d516e2850e37a083cc2fbe7079b9167c9d6ded17e89a64313aaf930
  Stored in directory: /tmp/pip-ephem-wheel-cache-fzkyyfae/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <cuda.h>
#include <device_functions.h>
#include <cuda_runtime_api.h>
#include "/content/pgmio.h"

// image dimensions WIDTH & HEIGHT
#define WIDTH 225
#define HEIGHT 225

// Block width WIDTH & HEIGHT
#define BLOCK_W 16
#define BLOCK_H 16

// buffer to read image into
float image[HEIGHT][WIDTH];

// buffer for resulting image
float final[HEIGHT][WIDTH];

// prototype declarations
void load_image();
void call_kernel();
void save_image();

#define MAXLINE 128

float total, sobel;
cudaEvent_t start_total, stop_total;
cudaEvent_t start_sobel, stop_sobel;

__global__ void imageBlur(float *input, float *output, int width, int height) {
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    int row = threadIdx.y + blockIdx.y * blockDim.y;

    int numcols = WIDTH;

    float blur;

    if (row < height && col < width && row > 0 && col > 0) {
        int x1, x3, x4, x5, x7;

        x1 = input[(row + 1) * numcols + col];
        x3 = input[row * numcols + (col - 1)];
        x4 = input[row * numcols + col];
        x5 = input[row * numcols + (col + 1)];
        x7 = input[(row - 1) * numcols + col];

        blur = (x1 * 0.2) + (x3 * 0.2) + (x4 * 0.2) + (x5 * 0.2) + (x7 * 0.2);

        output[row * numcols + col] = blur;
    }
}

__global__ void gaussianFilter(float *input, float *output, int width, int height) {
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    int row = threadIdx.y + blockIdx.y * blockDim.y;

    int numcols = WIDTH;

    float blur;

    if (row < height && col < width && row > 0 && col > 0) {
        int x0, x1, x2, x3, x4, x5, x6, x7, x8;

        x0 = input[(row - 1) * numcols + (col - 1)];
        x1 = input[(row - 1) * numcols + col];
        x2 = input[(row - 1) * numcols + (col + 1)];
        x3 = input[row * numcols + (col - 1)];
        x4 = input[row * numcols + col];
        x5 = input[row * numcols + (col + 1)];
        x6 = input[(row + 1) * numcols + (col - 1)];
        x7 = input[(row + 1) * numcols + col];
        x8 = input[(row + 1) * numcols + (col + 1)];

        blur = (x0 + 2 * x1 + x2 + 2 * x3 + 4 * x4 + 2 * x5 + x6 + 2 * x7 + x8) / 16.0;



        output[row * numcols + col] = blur;
    }
}

void load_image() {
    pgmread("/content/image225x225.pgm", (void *)image, WIDTH, HEIGHT);
}

void save_image() {
    pgmwrite("/content/image-outputl225x225.pgm", (void *)final, WIDTH, HEIGHT);
}

void call_kernel() {
    int x, y;
    float *d_input, *d_output;

    printf("Block size: %dx%d\n", BLOCK_W, BLOCK_H);

    size_t memSize = WIDTH * HEIGHT * sizeof(float);

    cudaMalloc(&d_input, memSize);
    cudaMalloc(&d_output, memSize);

    for (y = 0; y < HEIGHT; y++) {
        for (x = 0; x < WIDTH; x++) {
            final[x][y] = 0.0;
        }
    }

    printf("Blocks per grid (width): %d |", (WIDTH / BLOCK_W));
    printf("Blocks per grid (height): %d |", (HEIGHT / BLOCK_H));

    cudaMemcpy(d_input, image, memSize, cudaMemcpyHostToDevice);

    dim3 threads(BLOCK_W, BLOCK_H);
    dim3 blocks(WIDTH / BLOCK_W, HEIGHT / BLOCK_H);

    imageBlur<<<blocks, threads>>>(d_input, d_output, WIDTH, HEIGHT);
    cudaDeviceSynchronize();

    cudaMemcpy(d_input, d_output, memSize, cudaMemcpyDeviceToHost);

    gaussianFilter<<<blocks, threads>>>(d_input, d_output, WIDTH, HEIGHT);
    cudaDeviceSynchronize();

    cudaMemcpy(final, d_output, memSize, cudaMemcpyDeviceToHost);

    cudaError_t err = cudaGetLastError();
    if (cudaSuccess != err) {
        fprintf(stderr, "Cuda error: %s: %s.\n", "Main Loop", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    cudaFree(d_input);
    cudaFree(d_output);
}

int main(int argc, char *argv[]) {
    cudaEventCreate(&start_total);
    cudaEventCreate(&stop_total);

    cudaEventCreate(&start_sobel);
    cudaEventCreate(&stop_sobel);

    cudaEventRecord(start_total, 0);

    load_image();

    cudaEventRecord(start_sobel, 0);

    call_kernel();

    cudaEventRecord(stop_sobel, 0);
    cudaEventSynchronize(stop_sobel);
    cudaEventElapsedTime(&sobel, start_sobel, stop_sobel);

    save_image();

    cudaEventRecord(stop_total, 0);
    cudaEventSynchronize(stop_total);
    cudaEventElapsedTime(&total, start_total, stop_total);

    printf("Total Parallel Time: %f s |", sobel / 1000);
    printf("Total Serial Time: %f s |", (total - sobel) / 1000);
    printf("Total Time: %f s |", total / 1000);

    cudaDeviceReset();

    return 0;
}


Block size: 16x16
Blocks per grid (width): 14 |Blocks per grid (height): 14 |Writing 225 x 225 picture into file: /content/image-outputl225x225.pgm
Total Parallel Time: 0.001644 s |Total Serial Time: 0.009759 s |Total Time: 0.011403 s |
